In [45]:
!pip install arxiv wikipedia

### Create tools

https://python.langchain.com/v0.1/docs/use_cases/tool_use/

##### Create wikipedia retriever tool

In [46]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper(
    top_k_results=1,
    doc_content_chars_max=200
)

wiki_tool = WikipediaQueryRun( api_wrapper=api_wrapper )

##### Create custom retriever tool

In [47]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com")
docs=loader.load()
documents = RecursiveCharacterTextSplitter( chunk_size=1000, chunk_overlap=200 ).split_documents(docs)
vectordb = FAISS.from_documents( documents, OpenAIEmbeddings() )
retriever = vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x1379d1150>)

In [48]:
from langchain.tools.retriever import create_retriever_tool
langsmith_tool = create_retriever_tool(retriever, "langsmith_search", "Search for information about LAngsmith. For any questions about Langsmith, you must use this tool.")

##### Create Arxiv tool

In [49]:
from langchain_community.tools import ArxivQueryRun
from langchain_community.utilities import ArxivAPIWrapper

arxiv_api_wrapper = ArxivAPIWrapper(
    top_k_results=1,
    doc_content_chars_max=200
)

arxiv_tool = ArxivQueryRun( api_wrapper=arxiv_api_wrapper )

##### Create Pubmed tool

In [50]:
from langchain_community.tools import PubmedQueryRun
from langchain_community.utilities import PubMedAPIWrapper

pubmed_api_wrapper = PubMedAPIWrapper(
    top_k_results=1,
    doc_content_chars_max=200
)

pubmed_tool = PubmedQueryRun( api_wrapper=pubmed_api_wrapper )

##### Create toolkit

In [51]:
tools = [wiki_tool, arxiv_tool, langsmith_tool, pubmed_tool]

### Agents

https://python.langchain.com/v0.1/docs/modules/agents/

```
The core idea of agents is to use a language model to choose a sequence of actions to take. In chains, a sequence of actions is hardcoded (in code). In agents, a language model is used as a reasoning engine to determine which actions to take and in which order.
```

##### Setup OpenAI

In [52]:
from dotenv import load_dotenv

load_dotenv()

import os
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)


##### Access prompt hub

In [53]:
from langchain import hub

prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

##### Create agent

In [54]:
from langchain.agents import create_openai_tools_agent

agent = create_openai_tools_agent(llm, tools, prompt)

In [55]:
agent
# pubmed agent tool to query pubmed

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a1655024b06afbd95d17449f21316291e0726f13dcfaf990cc0d18087ad689a5'}, mess

##### Create agent executor

In [56]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a165

##### Invoke agent executor

In [57]:
agent_executor.invoke({ "input": "What is Langsmith used for?"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'What is Langsmith used for?'}`


Get started with LangSmith | 🦜️🛠️ LangSmith

Skip to main contentGo to API DocsSearchGo to AppQuick startTutorialsHow-to guidesConceptsReferencePricingSelf-hostingLangGraph CloudQuick startOn this pageGet started with LangSmithLangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!1. Install LangSmith​PythonTypeScriptpip install -U langsmithyarn add langchain langsmith2. Create an API key​To create an API key head to the Settings page. Then click Create API Key.3. Set up your environment​Shellexport LANGCHAIN_TRACING_V2=trueexport LANGCHAIN_API_KEY=<your-api-key># The below examples use the OpenAI API, though it's not necessary in generalexport OPENAI_API_KEY=<your-openai-api-key>4.

{'input': 'What is Langsmith used for?',
 'output': 'LangSmith is a platform for building production-grade LLM (Large Language Model) applications. It allows you to closely monitor and evaluate your application, enabling you to ship quickly and with confidence. LangSmith works independently, and the use of LangChain is not necessary for its operation. To get started with LangSmith, you can follow these steps:\n\n1. Install LangSmith using Python or TypeScript.\n2. Create an API key by heading to the Settings page and clicking on Create API Key.\n3. Set up your environment by exporting necessary variables like LANGCHAIN_TRACING_V2 and LANGCHAIN_API_KEY.\n4. Log your first trace using the provided methods in LangSmith.\n\nLangSmith provides tools for creating datasets, examples, evaluators, and running evaluations to enhance the performance of your LLM applications.'}

In [58]:
agent_executor.invoke({ "input": "What is Michael Jordan known for?"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Michael Jordan'}`


Page: Michael Jordan
Summary: Michael Jeffrey Jordan (born February 17, 1963), also known by his initials MJ, is an American businessman and former professional basketball player. He played fifteen seMichael Jordan is known for being an American businessman and former professional basketball player. He is widely regarded as one of the greatest basketball players of all time.

> Finished chain.


{'input': 'What is Michael Jordan known for?',
 'output': 'Michael Jordan is known for being an American businessman and former professional basketball player. He is widely regarded as one of the greatest basketball players of all time.'}

In [59]:
agent_executor.invoke({ "input": "What is the role of epigenetics in cancer?"})




> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Epigenetics in cancer'}`


Page: Cancer epigenetics
Summary: Cancer epigenetics is the study of epigenetic modifications to the DNA of cancer cells that do not involve a change in the nucleotide sequence, but instead involve a Epigenetics in cancer refers to the study of epigenetic modifications to the DNA of cancer cells that do not involve changes in the nucleotide sequence. These modifications can affect gene expression and play a significant role in the development and progression of cancer. Epigenetic changes in cancer cells can lead to the activation of oncogenes or the silencing of tumor suppressor genes, contributing to the uncontrolled growth and spread of cancer cells.

> Finished chain.


{'input': 'What is the role of epigenetics in cancer?',
 'output': 'Epigenetics in cancer refers to the study of epigenetic modifications to the DNA of cancer cells that do not involve changes in the nucleotide sequence. These modifications can affect gene expression and play a significant role in the development and progression of cancer. Epigenetic changes in cancer cells can lead to the activation of oncogenes or the silencing of tumor suppressor genes, contributing to the uncontrolled growth and spread of cancer cells.'}

In [60]:
agent_executor.invoke({ "input": "Are there any papers that explain how can you quantify macroeconomic expectations in stock markets using Google Trends?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': 'quantifying macroeconomic expectations in stock markets using Google Trends'}`


Published: 2018-05-01
Title: Quantifying macroeconomic expectations in stock markets using Google Trends
Authors: Johannes Bock
Summary: Among other macroeconomic indicators, the monthly release of U.I found a paper titled "Quantifying macroeconomic expectations in stock markets using Google Trends" by Johannes Bock. The paper discusses how Google Trends can be used to quantify macroeconomic expectations in stock markets.

> Finished chain.


{'input': 'Are there any papers that explain how can you quantify macroeconomic expectations in stock markets using Google Trends?',
 'output': 'I found a paper titled "Quantifying macroeconomic expectations in stock markets using Google Trends" by Johannes Bock. The paper discusses how Google Trends can be used to quantify macroeconomic expectations in stock markets.'}

In [61]:
agent_executor.invoke({ "input": "Are there any papers that explain the role of epigenetic factors in chronic smokers and cancer?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': 'epigenetic factors in chronic smokers and cancer'}`


Published: 2023-09-26
Title: Statistical Applications in Pain Management: A Review
Authors: Flora Fan
Summary: Chronic pain is a widespread and debilitating condition that affects millions
of individu
Invoking: `pub_med` with `epigenetic factors in chronic smokers and cancer`


Published: 2023-10-14
Title: Extracellular vesicles from subjects with COPD modulate cancer initiating cells phenotype through HIF-1α shuttling.
Copyright Information: © 2023. The Author(s).
Summary::I couldn't find specific papers on the role of epigenetic factors in chronic smokers and cancer. However, I can provide you with some general information on the topic. Would you like me to do that?

> Finished chain.


{'input': 'Are there any papers that explain the role of epigenetic factors in chronic smokers and cancer?',
 'output': "I couldn't find specific papers on the role of epigenetic factors in chronic smokers and cancer. However, I can provide you with some general information on the topic. Would you like me to do that?"}